In [1]:
import pandas as pd
import wrds

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.cusip
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        and b.date <= '12/29/2023'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [5]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ticker, permno
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

In [11]:
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')
sp500_full.rename(columns={'date':'datadate'},inplace=True)
sp500_full.sample(5)

,permno,start,ending,datadate,cusip,comnam,ticker
67709,21792,1985-07-25,2023-12-29,2001-07-31,15189T10,RELIANT ENERGY INC TX,REI
479683,26403,1976-07-01,2023-12-29,2010-06-30,25468710,DISNEY WALT CO,DIS
152017,15720,1925-12-31,2023-12-29,2003-05-30,28102010,EDISON INTERNATIONAL,EIX
76082,45495,1985-01-10,2014-08-18,2001-09-28,G7665A10,ROWAN DRILLING CO,RDC
147566,40539,1985-09-26,2023-12-29,2003-04-30,87254010,T J X COMPANIES INC NEW,TJX


In [37]:
#conn.list_tables(library='comp')

In [31]:
income = conn.raw_sql("""select datadate, cusip, tic, conm, revt, xsga, ebit, dp, ni
                        from comp.funda
                        where datadate >= '01/01/2000'""",
                        date_cols=['datadate'])

In [34]:
income.dropna(inplace=True)
income.rename(columns={'tic':'ticker','conm':'company.name'},inplace=True)
income.sample(5)

,datadate,cusip,ticker,company.name,revt,xsga,ebit,dp,ni
324812,2015-12-31,706327103,PBA,PEMBINA PIPELINE CORP,4635.000,143.000,665.000,272.000,406.000
454707,2020-12-31,89102U103,TRCH,TORCHLIGHT ENERGY RESOURCES,0.193,3.527,-6.451,0.821,-12.782
256654,2004-12-31,96808B107,OATS,WILD OATS MARKETS INC,1048.164,301.744,-4.894,28.059,-40.019
29319,2011-03-31,44949K107,IEHC,IEH CORP,13.824,1.984,2.986,0.161,1.670
29996,2010-12-31,74624H103,PSWS,PURESAFE WATER SYSTEMS INC,0.000,3.499,-3.540,0.041,-6.640


In [40]:
#TODO: figure out how to fucking merge the two together
conn.close()